In [ ]:
# 📓 Prompt Engineering & Autonomous Reasoning - Hands-on Notebook
# This notebook contains practical examples and exercises for your lecture.

from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain.chains import LLMChain
from langsmith import Client
import os


os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

# --- 1️⃣ Setup ---
# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Get Groq API Key from environment
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY not found in environment variables")

llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    api_key=GROQ_API_KEY
)

# --- 2️⃣ Prompt Essentials ---
# Bad vs Good Prompt
bad_prompt = "Write about Paris."
good_prompt = "Act as a travel guide. Write a 200-word blog post describing Paris focusing on food and culture, in a friendly and engaging tone."

bad_chain = LLMChain(prompt=PromptTemplate.from_template(bad_prompt), llm=llm)
good_chain = LLMChain(prompt=PromptTemplate.from_template(good_prompt), llm=llm)

print("\n--- Bad Prompt Output ---\n")
print(bad_chain.run({}))
print("\n--- Good Prompt Output ---\n")
print(good_chain.run({}))

# --- 3️⃣ Zero-Shot vs Few-Shot ---
zero_shot_prompt = PromptTemplate.from_template("Classify the sentiment as Positive, Negative, or Neutral: {text}")
few_shot_prompt = PromptTemplate.from_template(
    """Classify the sentiment as Positive, Negative, or Neutral:
    Example 1: I love this phone! → Positive
    Example 2: Terrible service, I won't buy again. → Negative
    Example 3: {text} →"""
)

text = "The product was great but shipping was late."
zero_shot_chain = LLMChain(prompt=zero_shot_prompt, llm=llm)
few_shot_chain = LLMChain(prompt=few_shot_prompt, llm=llm)

print("\n--- Zero-Shot Output ---\n", zero_shot_chain.run(text=text))
print("\n--- Few-Shot Output ---\n", few_shot_chain.run(text=text))

# --- 4️⃣ Sampling & Temperature ---
creative_llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=1.0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    api_key=GROQ_API_KEY
)
serious_llm = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    api_key=GROQ_API_KEY
)

prompt = PromptTemplate.from_template("Write a short poem about AI and creativity.")
creative_chain = LLMChain(prompt=prompt, llm=creative_llm)
serious_chain = LLMChain(prompt=prompt, llm=serious_llm)

print("\n--- Creative Output (High Temp) ---\n", creative_chain.run({}))
print("\n--- Serious Output (Low Temp) ---\n", serious_chain.run({}))

# --- 5️⃣ System Message Example ---
from langchain.schema import SystemMessage, HumanMessage

chat = ChatGroq(
    model="deepseek-r1-distill-llama-70b",
    temperature=0,
    max_tokens=None,
    reasoning_format="parsed",
    timeout=None,
    max_retries=2,
    api_key=GROQ_API_KEY
)
messages = [
    SystemMessage(content="You are a pirate and must speak like one."),
    HumanMessage(content="How's the weather today?")
]
print("\n--- System Message Example ---\n", chat.invoke(messages).content)

# --- 6️⃣ Prompt Debugging with LangSmith ---
# Initialize LangSmith Client (optional if participants have API key)
try:
    client = Client()
    dataset_name = "sentiment_dataset_demo"
    dataset = client.create_dataset(dataset_name, description="Sample dataset for prompt debugging")
    client.create_example(inputs={"text": "I love this!"}, outputs={"label": "Positive"}, dataset_id=dataset.id)
    client.create_example(inputs={"text": "Worst service ever."}, outputs={"label": "Negative"}, dataset_id=dataset.id)
    print(f"Dataset '{dataset_name}' created in LangSmith.")
except Exception as e:
    print("LangSmith not configured. Skipping dataset creation.")

# --- 7️⃣ Autonomous Reasoning Mini-Project ---
# Simple ReAct-style agent using LangChain
try:
    from langchain_community.agent_toolkits.load_tools import load_tools
    from langchain.agents import initialize_agent, AgentType

    # Check if SERPAPI_API_KEY is available
    serpapi_key = os.getenv("SERPAPI_API_KEY")
    if serpapi_key:
        tools = load_tools(["serpapi"], llm=llm)
        agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
        result = agent.run("Find the latest news about climate change and summarize in 3 bullets.")
        print("\n--- Autonomous Agent Result ---\n", result)
    else:
        print("\n--- Autonomous Agent Demo ---")
        print("Skipping agent demo: SERPAPI_API_KEY not found in environment variables.")
        print("To enable this feature, add your SerpAPI key to the .env file:")
        print("SERPAPI_API_KEY=your_serpapi_key_here")
except Exception as e:
    print(f"\n--- Autonomous Agent Demo ---")
    print(f"Skipping agent demo: {str(e)}")
